In [1]:
import torch
import numpy as np
import re
from transformers import AutoTokenizer, AutoModel
from nltk import word_tokenize

In [2]:
#for reproducibility
seed=20
cust_seed = np.random.seed(seed)

In [596]:
tokenizer = AutoTokenizer.from_pretrained("vblagoje/bert-english-uncased-finetuned-pos")
model = AutoModel.from_pretrained("vblagoje/bert-english-uncased-finetuned-pos")

#tokenizer.add_special_tokens({'pad_token': '[PAD]'})
#model.resize_token_embeddings(len(tokenizer))

Some weights of the model checkpoint at vblagoje/bert-english-uncased-finetuned-pos were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [687]:
def set_creator(chunk):
    text = []
    tag_counter = 0
    for idx3, k in enumerate(data):
        for idx4, m in enumerate(k):
            if chunk*40 <= tag_counter < (chunk+1)*40:
                text.append(m[0])
            tag_counter += 1
    text = ' '.join(text)
    input_string = text
    
    pattern = '([:;.,!?()])'
    input_string = re.sub(pattern, r' \1 ', input_string)
    input_string = re.sub('\s\s*',' ',input_string)
    input_string = re.sub(" '","'",input_string)
    input_string = re.sub("s' ","s ' ",input_string)
    
    return input_string   
#print(set_creator(4))

In [600]:
#ref: https://discuss.huggingface.co/t/generate-raw-word-embeddings-using-transformer-models-like-bert-for-downstream-process/2958/2

def get_hidden_states(encoded, token_ids_word, model, layers):
    """Push input IDs through model. Stack and sum 'layers' (last four by default).
       Select only those subword token outputs that belong to our word of interest
       and average them."""
    with torch.no_grad():
        output = model(**encoded)
        #print(output)
    # Get all hidden states
    states = output.hidden_states
    # Stack and sum all requested layers
    output = torch.stack([states[o] for o in layers]).sum(0).squeeze()
    print('output',np.shape(output))
    # Only select the tokens that constitute the requested word
    word_tokens_output = output[token_ids_word]
    print('word_tokens_output',word_tokens_output)
    print('avg subcomp word',word_tokens_output.mean(dim=0))
    return word_tokens_output.mean(dim=0) # this averages the sub components of the word

def get_word_vector(sent, idx, tokenizer, model, layers):
    """Get a word vector by first tokenizing the input sentence, getting all token idxs
       that make up the word of interest, and then 'get_hidden_states'."""
    encoded = tokenizer.encode_plus(sent, return_tensors="pt")
    print('encoded',encoded)
    print('encoded.word_ids()',encoded.word_ids())
    # get all token idxs that belong to the word of interest
    token_ids_word = np.where(np.array(encoded.word_ids()) == idx)
    print('token_ids_word',token_ids_word)
    print('encoded.word_ids-idx',(np.array(encoded.word_ids()) == idx),(encoded.word_ids()) == idx)
    return get_hidden_states(encoded, token_ids_word, model, layers)

#def main(layers=None):
    #Use last four layers by default
    #layers = [-4, -3, -2, -1] if layers is None else layers
    #tokenizer = AutoTokenizer.from_pretrained("gpt2")
   # model = AutoModel.from_pretrained("gpt2", output_hidden_states=True)

    #sent = "I like cookies ." 
    #idx = get_word_idx(sent, "cookies")

    #word_embedding = get_word_vector(sent, idx, tokenizer, model, layers)
    
   # return word_embedding 

#if __name__ == '__main__':
#    main()

In [929]:
#ref: https://discuss.huggingface.co/t/generate-raw-word-embeddings-using-transformer-models-like-bert-for-downstream-process/2958/2
def get_word_idx(sent: str, word: str):
    return sent.split(" ").index(word)


def get_hidden_states(encoded, token_ids_word, model, layers):
    """Push input IDs through model. Stack and sum `layers` (last four by default).
       Select only those subword token outputs that belong to our word of interest
       and average them."""
    with torch.no_grad():
        output = model(**encoded)
    # Get all hidden states
    states = output.hidden_states
    # Stack and sum all requested layers
    output = torch.stack([states[i] for i in layers]).sum(0).squeeze()
    # Only select the tokens that constitute the requested word
    word_tokens_output = output[token_ids_word]

    return word_tokens_output.mean(dim=0) # this averages the sub components of the word


def get_word_vector(sent, idx, tokenizer, model, layers):
    """Get a word vector by first tokenizing the input sentence, getting all token idxs
       that make up the word of interest, and then `get_hidden_states`."""
    encoded = tokenizer.encode_plus(sent, return_tensors="pt")
    # get all token idxs that belong to the word of interest
    token_ids_word = np.where(np.array(encoded.word_ids()) == idx)

    return get_hidden_states(encoded, token_ids_word, model, layers)


def main(layers=None):
    #Use last four layers by default
    layers = [-4, -3, -2, -1] if layers is None else layers
    tokenizer = AutoTokenizer.from_pretrained("vblagoje/bert-english-uncased-finetuned-pos")
    model = AutoModel.from_pretrained("vblagoje/bert-english-uncased-finetuned-pos", output_hidden_states=True)

    #sent = "I like cookies ." 
    #idx = get_word_idx(sent, "cookies")

    #word_embedding = get_word_vector(sent, idx, tokenizer, model, layers)
    
   # return word_embedding 


if __name__ == '__main__':
    main()

Some weights of the model checkpoint at vblagoje/bert-english-uncased-finetuned-pos were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [680]:
#Here I'm trying to pad the string so that the punctuation is separated when grabbing the embedding
# if the string isn't padded, it'll try to find 'pumpkin.' instead of 'pumpkin', '.'
import re
pattern = '([:;.,!?()])'
input_string = re.sub(pattern, r' \1 ', input_string)
input_string = re.sub('\s\s*',' ',input_string)
input_string = re.sub(" '","'",input_string)

In [681]:
import tensorflow as tf
import nltk.corpus 
import nltk
nltk.download('mte_teip5')

[nltk_data] Downloading package mte_teip5 to
[nltk_data]     /Users/Mahwish/nltk_data...
[nltk_data]   Package mte_teip5 is already up-to-date!


True

In [30]:
st = 'yo my name is blah and im tokenizing this stuff'
#tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
#tokenizer('st')
st.split(" ").index('blah')
st.split(" ")
ww[0]

'caption'

In [682]:
# ----------- I ----------- #
#train_data = nltk.corpus.multext_east.words("oana-en.xml") 
data = nltk.corpus.multext_east.tagged_sents("oana-en.xml", "universal")

In [684]:
#ww

In [1272]:
# Use last four layers by default
counter=0
layers = [-4, -3, -2, -1]
tokenizer = AutoTokenizer.from_pretrained("vblagoje/bert-english-uncased-finetuned-pos")
model = AutoModel.from_pretrained("vblagoje/bert-english-uncased-finetuned-pos", output_hidden_states=True)
tens = torch.empty((0), dtype=torch.int32)
extra_set_list = []
for q in range(0,400):
    input_string = set_creator(q)
    ww = input_string.split(" ")
    print('q',q)

    for ix, i in enumerate(ww):
        print(ix,i)
        #print(list_labels[ix])
        if i != '':
            extra_set_list.append(i)
           # idx = get_word_idx(input_string,ww[ix])
            idx = ix
            word_embedding = get_word_vector(input_string,idx,tokenizer,model,layers)
            #if (ix == 0 and q == 0): # CHANGE THIS 
            #    tens = word_embedding[None,:]
            #    print('tens shape',np.shape(tens))
            #else:
            tens = torch.cat((tens, word_embedding[None,:]),0)
            print(ix,i, 'tens shape',np.shape(tens))
            print(word_embedding[0])
            print('counter',counter)
            counter+=1


Some weights of the model checkpoint at vblagoje/bert-english-uncased-finetuned-pos were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


q 0
0 It
0 It tens shape torch.Size([1, 768])
tensor(1.7538)
counter 0
1 was
1 was tens shape torch.Size([2, 768])
tensor(0.5775)
counter 1
2 a
2 a tens shape torch.Size([3, 768])
tensor(-4.3772)
counter 2
3 bright
3 bright tens shape torch.Size([4, 768])
tensor(-0.7954)
counter 3
4 cold
4 cold tens shape torch.Size([5, 768])
tensor(0.6912)
counter 4
5 day
5 day tens shape torch.Size([6, 768])
tensor(2.5300)
counter 5
6 in
6 in tens shape torch.Size([7, 768])
tensor(-4.1366)
counter 6
7 April
7 April tens shape torch.Size([8, 768])
tensor(1.4610)
counter 7
8 ,
8 , tens shape torch.Size([9, 768])
tensor(0.4316)
counter 8
9 and
9 and tens shape torch.Size([10, 768])
tensor(-1.8758)
counter 9
10 the
10 the tens shape torch.Size([11, 768])
tensor(-3.8996)
counter 10
11 clocks
11 clocks tens shape torch.Size([12, 768])
tensor(4.2557)
counter 11
12 were
12 were tens shape torch.Size([13, 768])
tensor(1.2797)
counter 12
13 striking
13 striking tens shape torch.Size([14, 768])
tensor(1.2858)
c

31 a tens shape torch.Size([111, 768])
tensor(3.1779)
counter 110
32 man
32 man tens shape torch.Size([112, 768])
tensor(-1.3872)
counter 111
33 of
33 of tens shape torch.Size([113, 768])
tensor(-0.6409)
counter 112
34 about
34 about tens shape torch.Size([114, 768])
tensor(2.5633)
counter 113
35 forty-five
35 forty-five tens shape torch.Size([115, 768])
tensor(1.3844)
counter 114
36 ,
36 , tens shape torch.Size([116, 768])
tensor(1.7509)
counter 115
37 with
37 with tens shape torch.Size([117, 768])
tensor(0.2284)
counter 116
38 a
38 a tens shape torch.Size([118, 768])
tensor(-1.9741)
counter 117
39 heavy
39 heavy tens shape torch.Size([119, 768])
tensor(-4.2893)
counter 118
40 black
40 black tens shape torch.Size([120, 768])
tensor(1.3117)
counter 119
q 3
0 moustache
0 moustache tens shape torch.Size([121, 768])
tensor(2.0940)
counter 120
1 and
1 and tens shape torch.Size([122, 768])
tensor(-3.8513)
counter 121
2 ruggedly
2 ruggedly tens shape torch.Size([123, 768])
tensor(0.0789)
cou

17 , tens shape torch.Size([218, 768])
tensor(0.4407)
counter 217
18 the
18 the tens shape torch.Size([219, 768])
tensor(5.9872)
counter 218
19 poster
19 poster tens shape torch.Size([220, 768])
tensor(0.2314)
counter 219
20 with
20 with tens shape torch.Size([221, 768])
tensor(-2.7663)
counter 220
21 the
21 the tens shape torch.Size([222, 768])
tensor(4.0354)
counter 221
22 enormous
22 enormous tens shape torch.Size([223, 768])
tensor(-1.5653)
counter 222
23 face
23 face tens shape torch.Size([224, 768])
tensor(-3.9915)
counter 223
24 gazed
24 gazed tens shape torch.Size([225, 768])
tensor(0.3560)
counter 224
25 from
25 from tens shape torch.Size([226, 768])
tensor(4.1014)
counter 225
26 the
26 the tens shape torch.Size([227, 768])
tensor(1.6581)
counter 226
27 wall
27 wall tens shape torch.Size([228, 768])
tensor(-2.8254)
counter 227
28 .
28 . tens shape torch.Size([229, 768])
tensor(-3.7290)
counter 228
29 It
29 It tens shape torch.Size([230, 768])
tensor(0.9682)
counter 229
30 was


3 distinguishable tens shape torch.Size([324, 768])
tensor(-1.3668)
counter 323
4 .
4 . tens shape torch.Size([325, 768])
tensor(0.2321)
counter 324
5 The
5 The tens shape torch.Size([326, 768])
tensor(-3.1766)
counter 325
6 instrument
6 instrument tens shape torch.Size([327, 768])
tensor(1.3303)
counter 326
7 (
7 ( tens shape torch.Size([328, 768])
tensor(-0.1504)
counter 327
8 the
8 the tens shape torch.Size([329, 768])
tensor(-2.8035)
counter 328
9 telescreen
9 telescreen tens shape torch.Size([330, 768])
tensor(2.8890)
counter 329
10 ,
10 , tens shape torch.Size([331, 768])
tensor(0.8457)
counter 330
11 it
11 it tens shape torch.Size([332, 768])
tensor(2.8583)
counter 331
12 was
12 was tens shape torch.Size([333, 768])
tensor(1.1438)
counter 332
13 called
13 called tens shape torch.Size([334, 768])
tensor(3.6134)
counter 333
14 )
14 ) tens shape torch.Size([335, 768])
tensor(0.1493)
counter 334
15 could
15 could tens shape torch.Size([336, 768])
tensor(3.2988)
counter 335
16 be
16 

...

23 its tens shape torch.Size([15458, 768])
tensor(0.0114)
counter 15457
24 engineers
24 engineers tens shape torch.Size([15459, 768])
tensor(2.6131)
counter 15458
25 ,
25 , tens shape torch.Size([15460, 768])
tensor(3.7747)
counter 15459
26 its
26 its tens shape torch.Size([15461, 768])
tensor(-2.0066)
counter 15460
27 producers
27 producers tens shape torch.Size([15462, 768])
tensor(3.2876)
counter 15461
28 ,
28 , tens shape torch.Size([15463, 768])
tensor(3.5058)
counter 15462
29 and
29 and tens shape torch.Size([15464, 768])
tensor(1.0145)
counter 15463
30 its
30 its tens shape torch.Size([15465, 768])
tensor(4.5035)
counter 15464
31 teams
31 teams tens shape torch.Size([15466, 768])
tensor(3.0866)
counter 15465
32 of
32 of tens shape torch.Size([15467, 768])
tensor(0.8434)
counter 15466
33 actors
33 actors tens shape torch.Size([15468, 768])
tensor(-1.4486)
counter 15467
34 specially
34 specially tens shape torch.Size([15469, 768])
tensor(3.3192)
counter 15468
35 chosen
35 chosen t

3 of tens shape torch.Size([15558, 768])
tensor(-2.9300)
counter 15557
4 the
4 the tens shape torch.Size([15559, 768])
tensor(-3.3195)
counter 15558
5 past
5 past tens shape torch.Size([15560, 768])
tensor(3.2191)
counter 15559
6 should
6 should tens shape torch.Size([15561, 768])
tensor(2.3760)
counter 15560
7 be
7 be tens shape torch.Size([15562, 768])
tensor(0.3320)
counter 15561
8 preserved
8 preserved tens shape torch.Size([15563, 768])
tensor(2.7342)
counter 15562
9 ,
9 , tens shape torch.Size([15564, 768])
tensor(0.2144)
counter 15563
10 that
10 that tens shape torch.Size([15565, 768])
tensor(-4.7206)
counter 15564
11 one
11 one tens shape torch.Size([15566, 768])
tensor(-2.8285)
counter 15565
12 falsified
12 falsified tens shape torch.Size([15567, 768])
tensor(0.7237)
counter 15566
13 ,
13 , tens shape torch.Size([15568, 768])
tensor(0.5115)
counter 15567
14 and
14 and tens shape torch.Size([15569, 768])
tensor(-4.3837)
counter 15568
15 the
15 the tens shape torch.Size([15570, 

25 spelling-book tens shape torch.Size([15659, 768])
tensor(-1.3685)
counter 15658
26 to
26 to tens shape torch.Size([15660, 768])
tensor(5.4677)
counter 15659
27 a
27 a tens shape torch.Size([15661, 768])
tensor(0.2187)
counter 15660
28 Newspeak
28 Newspeak tens shape torch.Size([15662, 768])
tensor(3.8025)
counter 15661
29 dictionary
29 dictionary tens shape torch.Size([15663, 768])
tensor(-0.7348)
counter 15662
30 .
30 . tens shape torch.Size([15664, 768])
tensor(-3.4869)
counter 15663
31 And
31 And tens shape torch.Size([15665, 768])
tensor(1.4044)
counter 15664
32 the
32 the tens shape torch.Size([15666, 768])
tensor(6.7810)
counter 15665
33 Ministry
33 Ministry tens shape torch.Size([15667, 768])
tensor(0.2832)
counter 15666
34 had
34 had tens shape torch.Size([15668, 768])
tensor(-1.3482)
counter 15667
35 not
35 not tens shape torch.Size([15669, 768])
tensor(-3.5831)
counter 15668
36 only
36 only tens shape torch.Size([15670, 768])
tensor(3.3689)
counter 15669
37 to
37 to tens s

4 kaleidoscope tens shape torch.Size([15758, 768])
tensor(2.8900)
counter 15757
5 known
5 known tens shape torch.Size([15759, 768])
tensor(0.6711)
counter 15758
6 as
6 as tens shape torch.Size([15760, 768])
tensor(-3.0945)
counter 15759
7 a
7 a tens shape torch.Size([15761, 768])
tensor(-2.1147)
counter 15760
8 versificator
8 versificator tens shape torch.Size([15762, 768])
tensor(0.1444)
counter 15761
9 .
9 . tens shape torch.Size([15763, 768])
tensor(0.0456)
counter 15762
10 There
10 There tens shape torch.Size([15764, 768])
tensor(0.9422)
counter 15763
11 was
11 was tens shape torch.Size([15765, 768])
tensor(-0.1718)
counter 15764
12 even
12 even tens shape torch.Size([15766, 768])
tensor(-1.0962)
counter 15765
13 a
13 a tens shape torch.Size([15767, 768])
tensor(-3.9409)
counter 15766
14 whole
14 whole tens shape torch.Size([15768, 768])
tensor(0.7072)
counter 15767
15 sub-section
15 sub-section tens shape torch.Size([15769, 768])
tensor(0.8909)
counter 15768
16 -
16 - tens shape t

26 , tens shape torch.Size([15859, 768])
tensor(-2.0238)
counter 15858
27 took
27 took tens shape torch.Size([15860, 768])
tensor(-2.6048)
counter 15859
28 the
28 the tens shape torch.Size([15861, 768])
tensor(0.6762)
counter 15860
29 Newspeak
29 Newspeak tens shape torch.Size([15862, 768])
tensor(6.3381)
counter 15861
30 Dictionary
30 Dictionary tens shape torch.Size([15863, 768])
tensor(-2.9460)
counter 15862
31 from
31 from tens shape torch.Size([15864, 768])
tensor(-4.7949)
counter 15863
32 the
32 the tens shape torch.Size([15865, 768])
tensor(3.0926)
counter 15864
33 shelf
33 shelf tens shape torch.Size([15866, 768])
tensor(1.2077)
counter 15865
34 ,
34 , tens shape torch.Size([15867, 768])
tensor(-0.2275)
counter 15866
35 pushed
35 pushed tens shape torch.Size([15868, 768])
tensor(-3.5977)
counter 15867
36 the
36 the tens shape torch.Size([15869, 768])
tensor(4.5181)
counter 15868
37 speakwrite
37 speakwrite tens shape torch.Size([15870, 768])
tensor(-1.9762)
counter 15869
38 to


In [932]:
# Use last four layers by default
layers = [-4, -3, -2, -1]
tokenizer = tokenizer #AutoTokenizer.from_pretrained("bert-base-cased")
model = model #AutoModel.from_pretrained("bert-base-cased", output_hidden_states=True)
tens = torch.empty((0), dtype=torch.int32)
extra_set_list = []
for q in range(0,10):
    input_string = set_creator(q)
    ww = input_string.split(" ")
    print('q',q)

    for ix, i in enumerate(ww):
        print(ix,i)
        extra_set_list.append(i)
       # idx = get_word_idx(input_string,ww[ix])
        idx = ix
        #print(idx,ix)
        #print(ww)
        word_embedding = get_word_vector(input_string,idx,tokenizer,model,layers)
        if (ix == 0 & q == 0): # CHANGE THIS 
            tens = word_embedding[None,:]
            print(word_embedding)
        else:
            tens = torch.cat((tens, word_embedding[None,:]),0)
            print('tens shape',np.shape(word_embedding))
            print(word_embedding)


q 0
0 It
tens shape torch.Size([768])
tensor([ 1.7538e+00, -4.0755e+00,  1.4447e-02,  5.9480e-01,  1.9120e-01,
         1.1187e+00,  1.8802e+00,  6.6475e+00,  4.9068e+00,  5.4331e-01,
        -1.2831e+00, -6.0172e-01, -1.7040e+00,  7.5291e-01,  1.9342e+00,
         5.6875e+00,  3.0025e+00,  3.1749e+00,  2.3344e+00, -2.9435e-01,
         1.6427e+00, -1.1979e+00,  4.9335e-01, -4.9814e-01, -2.7067e-01,
        -1.9250e+00, -2.4319e-01,  3.7911e-01, -2.9498e+00,  3.2545e+00,
        -9.1278e-01, -1.4441e+00,  2.7065e+00, -1.9520e+00,  1.1566e+00,
        -2.1656e+00, -1.4107e+00, -1.4889e+00, -6.1995e-01,  3.4011e+00,
        -6.9012e-02,  2.0331e-01, -2.3294e+00, -2.8671e+00, -1.0692e+00,
         2.1212e+00,  3.7559e+00,  2.8851e+00,  1.9222e+00,  1.3686e-01,
        -2.7229e+00,  2.4236e+00, -4.0017e+00, -2.0715e+00,  7.2687e-01,
        -3.5663e+00, -1.2630e+00, -7.1586e-01, -8.3904e-01,  3.9555e+00,
         4.9601e-01, -5.6036e+00,  4.1159e+00,  1.0545e+00, -2.6512e+00,
         5.96

tens shape torch.Size([768])
tensor([ 5.7749e-01, -2.4592e+00,  6.8357e-01,  5.8386e-02,  1.6274e-01,
         3.8192e-01,  1.7105e+00,  4.3829e+00,  3.2553e+00, -3.9502e+00,
        -1.1233e+00, -1.8695e+00,  4.9897e-01,  2.2335e+00, -2.8215e+00,
        -6.9319e-01, -2.4532e+00, -1.7450e+00,  5.5214e+00, -1.7424e-01,
        -9.0052e-01,  1.2659e+00, -9.8423e-01,  1.3452e+00,  2.0497e+00,
         6.0802e-02, -1.8915e+00,  2.6091e+00, -3.4945e+00,  3.7908e+00,
         2.4444e+00, -5.0780e-02,  2.9409e+00, -2.8523e+00,  6.4769e-01,
        -3.2779e+00,  1.0019e+00,  1.8029e+00, -4.6461e+00, -1.3528e+00,
        -1.0505e+00, -6.2684e+00, -2.0826e+00,  1.1008e+00, -2.8384e+00,
        -2.7156e-01,  2.8172e+00,  3.4847e+00,  2.8316e+00,  2.0064e+00,
        -3.1050e+00,  2.3165e+00,  2.9726e+00, -1.7820e-02, -2.0394e-01,
        -1.8265e+00, -2.2487e+00,  1.2712e+00, -1.5779e-01, -1.7721e+00,
         8.5482e-02, -3.2094e+00,  1.1089e+00,  7.2504e-01,  1.0290e+00,
         1.4380e+00, -

tens shape torch.Size([768])
tensor([-4.3772e+00, -9.3842e-01,  5.0152e-01,  3.3021e+00,  1.8722e+00,
        -1.8062e+00, -3.6733e-01,  2.3705e+00, -2.6504e+00,  1.3183e+00,
        -1.3148e+00,  1.8767e+00, -1.9399e+00,  1.8246e+00, -1.3891e+00,
         3.0906e-01, -7.5265e-01, -5.4750e-01,  2.1335e+00,  2.5573e+00,
        -4.9468e+00, -4.4380e+00, -2.3827e+00,  3.4347e+00,  6.0357e-01,
         6.8696e-01,  5.4791e-01,  3.5318e+00, -6.8993e-01, -2.5254e+00,
        -3.3334e+00,  1.4999e+00, -2.1647e+00, -2.0153e-01,  2.4825e+00,
        -3.1269e+00,  2.3177e-02, -2.9041e+00,  1.0821e+00,  3.8302e-01,
         5.9785e-01, -1.0447e-01, -6.8819e-01, -2.2801e+00, -4.8664e+00,
        -2.6684e+00, -1.4041e-01, -2.3533e+00,  2.4753e+00, -3.9204e+00,
         1.8632e+00, -8.5334e-01,  1.2313e+00, -8.1568e-01, -3.0886e+00,
        -2.1000e+00, -9.5191e-01, -3.5892e+00,  1.0993e+00,  8.7983e-01,
        -1.2378e+00,  5.8311e-01,  1.3416e+00, -4.8009e-01, -4.5092e-01,
        -3.5129e+00, -

tens shape torch.Size([768])
tensor([ -0.7954,  -0.7029,   3.0914,   2.6279,  -0.2637,   1.4710,  -2.8479,
          0.1046,  -0.4513,   2.4138,  -2.0897,  -3.1280,  -0.8221,   1.0432,
         -0.7202,   5.0839,   0.8890,   2.4654,  -4.3678,  -2.4163,  -3.1832,
          2.2013,   0.9130,   1.0816,  -0.1884,   2.9742,   6.2004,   0.3967,
         -1.8590,  -1.4818,   5.9223,  -1.0033,   1.9743,   0.3064,  -0.2121,
         -5.6175,  -0.1679,  -2.4256,  -0.7915,  -0.3783,   3.7074,  -3.0186,
          0.6316,   3.6076,   0.3667,  -0.2597,   0.0798,  -2.3694,   4.8381,
         -5.8650,  -1.3698,  -0.9115,   4.4399,   0.9989,  -1.3957,   2.4345,
         -1.4464,  -0.7360,   0.9787,  -3.8996,  -1.0700,   3.0887,   2.1923,
         -0.9566,   5.2199,  -1.0082,  -1.7376,  -3.6939,  -5.3715,   1.9966,
          3.2441,  -2.7789,  -4.8642,  -2.0367,  -1.5138,  -1.0572,  -1.5349,
         -0.6863,  -3.1313,  -1.5184,  -1.2667,   1.2771,   2.8281,   2.2984,
          1.2130,  -3.6440,   3.990

tens shape torch.Size([768])
tensor([ 6.9116e-01, -4.5925e-01,  2.7674e+00,  4.7709e+00, -4.5097e-01,
         7.1749e-01, -4.8716e-01,  3.1967e+00, -1.6570e+00,  2.9557e+00,
        -1.5827e+00, -2.5821e+00, -2.8986e+00,  3.6970e+00, -1.6957e+00,
         3.5195e+00,  1.9038e+00,  2.2326e+00, -2.2463e+00, -4.8631e+00,
        -2.6185e+00,  1.3394e+00,  5.5618e+00,  1.7603e+00,  2.4216e+00,
         5.1259e+00,  2.9424e+00,  2.2401e-01, -4.5378e+00, -1.5508e+00,
         9.3912e-01,  1.0563e+00,  5.6437e-01,  3.7330e+00,  3.3067e-01,
        -4.5326e+00,  1.4281e-01,  6.4521e-01, -1.7323e-01, -1.0763e+00,
         2.6681e+00, -1.0635e+00, -1.3926e+00,  3.4540e+00, -1.9107e-01,
         5.7779e-01,  1.2427e+00, -1.3700e+00,  2.5376e+00, -5.4499e+00,
         7.8817e-01, -5.3677e-01,  5.0645e+00, -1.2273e-01, -1.9904e+00,
         3.8840e+00, -1.8386e-02, -1.3370e+00, -2.9167e-01, -5.8153e+00,
         8.6751e-01,  3.8476e+00,  1.8345e+00, -3.1041e+00,  5.0208e+00,
         2.9539e+00, -

...

tens shape torch.Size([768])
tensor([-2.5549e+00,  1.1757e+00,  3.7476e+00, -1.0231e+00,  9.5175e-01,
         1.9197e-01,  5.9470e-01,  9.1237e-01,  2.7688e+00,  3.0599e+00,
         1.6049e+00,  1.3395e+00, -8.7135e-01,  4.1641e+00, -7.2512e-01,
         2.0394e+00,  2.6480e+00, -7.5992e-01,  2.0281e+00,  5.5672e+00,
        -7.1252e-02, -2.3867e+00, -1.5635e+00, -7.9964e-01,  1.0178e-01,
        -2.4043e+00,  9.0586e-01,  8.7092e-01,  8.5396e-01, -3.4180e+00,
         7.6071e+00,  3.4649e+00,  6.8028e+00, -2.3956e+00, -7.0041e-01,
        -3.5209e+00,  2.0823e+00, -5.2440e+00,  1.5221e+00,  3.5537e+00,
         4.7094e+00, -3.7744e+00, -1.5348e+00,  1.1748e+00, -1.6832e+00,
        -1.7752e+00, -1.1430e+00, -2.5223e+00,  1.6452e+00,  4.8288e-02,
         4.4214e+00, -1.1254e+00,  2.3767e+00,  3.3772e-01, -5.4918e-01,
         1.2508e+00, -6.8390e-01,  1.9534e+00,  4.7231e+00, -4.5549e+00,
        -2.5521e+00,  8.6741e-02, -1.2881e+00,  1.8349e+00, -1.7709e+00,
        -1.4954e+00,  

tens shape torch.Size([768])
tensor([-3.7420e+00, -2.6342e+00,  2.0060e+00,  1.6981e+00, -4.0763e-01,
        -3.3768e+00, -3.2135e+00,  1.4279e+00, -3.5405e+00,  2.3317e-01,
        -1.6991e+00,  4.3614e+00, -5.0967e+00,  4.0218e-01, -1.5031e+00,
         3.7938e+00,  1.9253e+00,  1.0642e+00,  1.6301e+00,  1.4671e+00,
        -4.5892e+00, -4.5636e+00,  4.6433e-01,  1.6258e+00,  2.0857e+00,
         2.3873e+00, -7.5167e-01,  1.0583e+00, -3.5750e+00, -1.6171e+00,
        -1.4978e+00, -8.8001e-01, -2.7862e+00, -1.9973e-01, -1.2527e+00,
        -3.7791e+00, -1.6421e+00, -2.2929e+00, -5.7442e-01, -1.2418e-01,
         3.1950e+00, -1.5554e+00, -1.0042e-01, -2.0791e+00, -3.4125e+00,
        -2.8314e+00, -8.6135e-01, -4.3397e+00, -1.8441e+00, -7.6129e+00,
         3.3138e-01, -5.1163e-01,  1.6037e+00,  2.1455e+00, -1.5899e+00,
         2.9139e+00,  1.3983e+00, -1.4676e+00,  2.5050e+00, -7.2955e-02,
        -4.6341e+00,  2.1405e+00,  5.3564e-01, -9.3200e-01,  1.1787e-01,
        -2.8545e+00,  

tens shape torch.Size([768])
tensor([ 3.8690e+00,  3.7420e-01,  3.9433e+00,  2.4829e+00, -2.2155e-01,
         6.2771e-01, -1.7196e+00,  1.6087e+00,  5.6159e-01, -1.9523e-01,
        -5.3160e-01, -1.8087e+00, -2.2895e+00,  5.4537e+00,  1.9860e+00,
         2.5380e+00,  3.1322e-01,  1.7594e+00, -6.4838e+00,  9.5853e-01,
        -4.1051e-01,  1.9435e+00,  3.9839e+00,  3.4550e-01,  2.1456e+00,
         3.8084e-01,  7.0709e-01,  2.8999e-01, -3.7385e+00, -3.5984e-02,
         4.8571e+00,  4.7070e+00,  3.6355e+00,  1.7054e+00, -2.4579e+00,
        -6.9982e+00,  2.8180e-01, -3.1615e+00, -1.9468e+00, -1.7657e+00,
         7.0662e+00, -3.5100e+00,  1.0909e+00,  1.5912e+00, -3.9892e-01,
         5.1391e-01, -4.4090e+00,  3.5887e-01,  8.6483e-02, -1.8250e+00,
        -9.4949e-01,  5.2443e-01, -1.2906e+00,  2.4855e+00,  7.2376e-01,
         5.6024e+00, -1.0856e-01, -1.7583e+00, -1.4080e+00, -2.5773e+00,
        -4.8565e-01,  3.6800e+00,  2.7365e+00, -2.7199e+00,  5.8396e+00,
        -3.3805e-01, -

tens shape torch.Size([768])
tensor([ 4.1760e+00, -1.0148e-01,  1.6436e+00, -1.8483e+00,  7.0834e-01,
        -1.0987e+00, -3.9062e+00,  2.4217e+00,  1.6284e+00, -9.6868e-01,
         1.8892e+00, -4.4887e-01, -1.0951e+00,  4.7533e+00,  2.4821e+00,
        -2.9426e-01, -1.6534e+00,  8.0178e-02, -4.2172e+00,  1.9920e+00,
         2.2447e+00,  7.9026e-01,  9.2630e-01, -1.1323e+00, -6.8351e-01,
        -3.3489e-01,  1.6211e+00,  2.4674e+00, -1.2336e+00, -8.3729e-01,
         4.0633e+00,  6.2390e+00,  4.5074e+00, -2.2153e+00, -3.9095e+00,
        -6.5450e+00,  4.9437e-03, -2.5366e+00,  2.2628e+00,  2.0633e+00,
         3.9180e+00, -2.6455e+00,  1.2343e+00, -3.0698e+00, -2.6519e+00,
         6.0010e-01, -1.9174e+00, -1.8017e+00, -2.3974e+00, -2.3546e+00,
         3.0511e-01, -4.3541e+00, -5.3939e+00,  2.5892e+00, -1.6782e+00,
         2.8653e+00,  2.0043e+00, -3.0870e+00, -2.7819e+00,  2.3462e-01,
         5.0761e-01,  2.8226e+00,  2.0398e+00, -1.8220e+00,  5.5671e+00,
         2.9228e+00, -

KeyboardInterrupt: 

#### The cell below works: Using this to create the embeddings in batches of 900. The above cell script does this dynamically, since the word embedding function above doesn't allow larger than 1000 len batch for input string.

### This part deals with POS tags data from NLTK corpus

## Label creators
I - Training labels
II - Test labels

In [1013]:
# list_labels=[]
# tag_counter = 0
# counter_label = 0
# for idx3, k in enumerate(data):
#     #print(idx3, k)
#     for idx4, m in enumerate(k):
#         if 0-1 < tag_counter < 40*200:
#             print(m)
#             list_labels.append(m)
#         tag_counter+=1

In [1551]:
## ----------- II ----------- #
#test label creator
bubu2=[]
dev_labels = []
tag_counter = 0
counter_label = 0
for idx3, k in enumerate(data):
    #print(idx3, k)
    for idx4, m in enumerate(k):
        #print(tag_counter,m)
        if (40*0)-1 < tag_counter < 40*401:
            if tag_counter in(5920,7200):
                print(tag_counter,m)
                bubu2.append("'s")
                dev_labels.append('VERB')
            elif tag_counter == 10760:
                bubu2.append("'s")
                dev_labels.append('ADP')
                print("s',ADP") 
            elif tag_counter == 8360:
                bubu2.append("'m")
                dev_labels.append('VERB')
                print("m,VERB")
#             elif tag_counter in (18858,23216):
#                 bubu2.append("B")
#                 dev_labels.append('NOUN')
#                 bubu2.append(".")
#                 dev_labels.append('')
#                 bubu2.append("B")
#                 dev_labels.append('NOUN')
#                 bubu2.append(".")
#                 dev_labels.append('')
            elif m[0] !=  "'s" and  m[0] != "'ve" and  m[0] != "'t" and m[0] !=  "'d" and m[0] !=  "'m" and m[0] !=  "'re" and m[0] !=  "'ave"and m[0] !=  "'ll" and m[0] !=  "'em" :
                #print(tag_counter,extra_set_list[tag_counter])
               # print('counter',counter_label,extra_set_list[tag_counter])
#                 if tag_counter in (7960,7880,7360,7160,6840,7000,7040,4600,4960,4880,5080,5240,5400,5640,5760,5800,6040,6120,6680,80,200,360,520,560,640,1000,1080,1320,1440,1480,1600,1640,1800,1880,2000,2200,2360,2400,2520,2640,3720,4000,4160,4320): #1600,13200,15200,17008,17010,17600,18400,18800,20400,21200,21600):
#                 #if int(counter_label) % 40 == 0 and extra_set_list[tag_counter] == '':   
#                     bubu2.append('')
#                     dev_labels.append('')
#                     print('')
#                     counter_label =counter_label-1
                if m[0] == '...':
                    bubu2.append('.')
                    dev_labels.append('')
                    bubu2.append('.')
                    dev_labels.append('')
                    print('') 
#                 elif tag_counter == 13297:
#                     bubu2.append('W')
#                     dev_labels.append('NOUN')
#                     print('WWW')   
#                 elif tag_counter in (13888,13895,13907,13913,16062):
#                     bubu2.append('17')
#                     dev_labels.append('NUM')
#                     bubu2.append('.')
#                     dev_labels.append('')
#                     bubu2.append('3')
#                     dev_labels.append('NUM')
#                     bubu2.append('.')
#                     dev_labels.append('')
                    
                bubu2.append(m[0])
                dev_labels.append(m[1])
                #print(ww[tag_counter])
            print(tag_counter,m)
        tag_counter += 1
        counter_label +=1

0 ('It', 'PRON')
1 ('was', 'VERB')
2 ('a', 'DET')
3 ('bright', 'ADJ')
4 ('cold', 'ADJ')
5 ('day', 'NOUN')
6 ('in', 'ADP')
7 ('April', 'NOUN')
8 (',', '')
9 ('and', 'CONJ')
10 ('the', 'DET')
11 ('clocks', 'NOUN')
12 ('were', 'VERB')
13 ('striking', 'VERB')
14 ('thirteen', 'NUM')
15 ('.', '')
16 ('Winston', 'NOUN')
17 ('Smith', 'NOUN')
18 (',', '')
19 ('his', 'DET')
20 ('chin', 'NOUN')
21 ('nuzzled', 'VERB')
22 ('into', 'ADP')
23 ('his', 'DET')
24 ('breast', 'NOUN')
25 ('in', 'ADP')
26 ('an', 'DET')
27 ('effort', 'NOUN')
28 ('to', 'ADP')
29 ('escape', 'VERB')
30 ('the', 'DET')
31 ('vile', 'ADJ')
32 ('wind', 'NOUN')
33 (',', '')
34 ('slipped', 'VERB')
35 ('quickly', 'ADV')
36 ('through', 'ADP')
37 ('the', 'DET')
38 ('glass', 'NOUN')
39 ('doors', 'NOUN')
40 ('of', 'ADP')
41 ('Victory', 'NOUN')
42 ('Mansions', 'NOUN')
43 (',', '')
44 ('though', 'CONJ')
45 ('not', 'ADV')
46 ('quickly', 'ADV')
47 ('enough', 'ADV')
48 ('to', 'ADP')
49 ('prevent', 'VERB')
50 ('a', 'DET')
51 ('swirl', 'NOUN')
52

669 ('seen', 'VERB')
670 ('as', 'CONJ')
671 ('well', 'ADV')
672 ('as', 'CONJ')
673 ('heard', 'VERB')
674 ('.', '')
675 ('There', 'PRON')
676 ('was', 'VERB')
677 ('of', 'ADP')
678 ('course', 'NOUN')
679 ('no', 'DET')
680 ('way', 'NOUN')
681 ('of', 'ADP')
682 ('knowing', 'VERB')
683 ('whether', 'CONJ')
684 ('you', 'PRON')
685 ('were', 'VERB')
686 ('being', 'VERB')
687 ('watched', 'VERB')
688 ('at', 'ADP')
689 ('any', 'DET')
690 ('given', 'VERB')
691 ('moment', 'NOUN')
692 ('.', '')
693 ('How', 'ADV')
694 ('often', 'ADV')
695 (',', '')
696 ('or', 'CONJ')
697 ('on', 'ADP')
698 ('what', 'DET')
699 ('system', 'NOUN')
700 (',', '')
701 ('the', 'DET')
702 ('Thought', 'NOUN')
703 ('Police', 'NOUN')
704 ('plugged', 'VERB')
705 ('in', 'ADP')
706 ('on', 'ADP')
707 ('any', 'DET')
708 ('individual', 'ADJ')
709 ('wire', 'NOUN')
710 ('was', 'VERB')
711 ('guesswork', 'NOUN')
712 ('.', '')
713 ('It', 'PRON')
714 ('was', 'VERB')
715 ('even', 'ADV')
716 ('conceivable', 'ADJ')
717 ('that', 'CONJ')
718 ('th

1512 ('belly', 'NOUN')
1513 ('died', 'VERB')
1514 ('down', 'ADV')
1515 ('and', 'CONJ')
1516 ('the', 'DET')
1517 ('world', 'NOUN')
1518 ('began', 'VERB')
1519 ('to', 'ADP')
1520 ('look', 'VERB')
1521 ('more', 'ADV')
1522 ('cheerful', 'ADJ')
1523 ('.', '')
1524 ('He', 'PRON')
1525 ('took', 'VERB')
1526 ('a', 'DET')
1527 ('cigarette', 'NOUN')
1528 ('from', 'ADP')
1529 ('a', 'DET')
1530 ('crumpled', 'ADJ')
1531 ('packet', 'NOUN')
1532 ('marked', 'VERB')
1533 ('Victory', 'NOUN')
1534 ('Cigarettes', 'NOUN')
1535 ('and', 'CONJ')
1536 ('incautiously', 'ADV')
1537 ('held', 'VERB')
1538 ('it', 'PRON')
1539 ('upright', 'ADV')
1540 (',', '')
1541 ('whereupon', 'ADV')
1542 ('the', 'DET')
1543 ('tobacco', 'NOUN')
1544 ('fell', 'VERB')
1545 ('out', 'ADP')
1546 ('on', 'ADP')
1547 ('to', 'ADP')
1548 ('the', 'DET')
1549 ('floor', 'NOUN')
1550 ('.', '')
1551 ('With', 'ADP')
1552 ('the', 'DET')
1553 ('next', 'NOUN')
1554 ('he', 'PRON')
1555 ('was', 'VERB')
1556 ('more', 'ADV')
1557 ('successful', 'ADJ')
1

2337 ('For', 'ADP')
2338 ('the', 'DET')
2339 ('first', 'NUM')
2340 ('time', 'NOUN')
2341 ('the', 'DET')
2342 ('magnitude', 'NOUN')
2343 ('of', 'ADP')
2344 ('what', 'PRON')
2345 ('he', 'PRON')
2346 ('had', 'VERB')
2347 ('undertaken', 'VERB')
2348 ('came', 'VERB')
2349 ('home', 'ADV')
2350 ('to', 'ADP')
2351 ('him', 'PRON')
2352 ('.', '')
2353 ('How', 'ADV')
2354 ('could', 'VERB')
2355 ('you', 'PRON')
2356 ('communicate', 'VERB')
2357 ('with', 'ADP')
2358 ('the', 'DET')
2359 ('future', 'NOUN')
2360 ('?', '')
2361 ('It', 'PRON')
2362 ('was', 'VERB')
2363 ('of', 'ADP')
2364 ('its', 'DET')
2365 ('nature', 'NOUN')
2366 ('impossible', 'ADJ')
2367 ('.', '')
2368 ('Either', 'CONJ')
2369 ('the', 'DET')
2370 ('future', 'NOUN')
2371 ('would', 'VERB')
2372 ('resemble', 'VERB')
2373 ('the', 'DET')
2374 ('present', 'NOUN')
2375 (',', '')
2376 ('in', 'ADP')
2377 ('which', 'DET')
2378 ('case', 'NOUN')
2379 ('it', 'PRON')
2380 ('would', 'VERB')
2381 ('not', 'ADV')
2382 ('listen', 'VERB')
2383 ('to', 'AD

3052 ('It', 'PRON')
3053 ('was', 'VERB')
3054 (',', '')
3055 ('he', 'PRON')
3056 ('now', 'ADV')
3057 ('realized', 'VERB')
3058 (',', '')
3059 ('because', 'CONJ')
3060 ('of', 'ADP')
3061 ('this', 'DET')
3062 ('other', 'DET')
3063 ('incident', 'NOUN')
3064 ('that', 'CONJ')
3065 ('he', 'PRON')
3066 ('had', 'VERB')
3067 ('suddenly', 'ADV')
3068 ('decided', 'VERB')
3069 ('to', 'ADP')
3070 ('come', 'VERB')
3071 ('home', 'ADV')
3072 ('and', 'CONJ')
3073 ('begin', 'VERB')
3074 ('the', 'DET')
3075 ('diary', 'NOUN')
3076 ('today', 'ADV')
3077 ('.', '')
3078 ('It', 'PRON')
3079 ('had', 'VERB')
3080 ('happened', 'VERB')
3081 ('that', 'DET')
3082 ('morning', 'NOUN')
3083 ('at', 'ADP')
3084 ('the', 'DET')
3085 ('Ministry', 'NOUN')
3086 (',', '')
3087 ('if', 'CONJ')
3088 ('anything', 'PRON')
3089 ('so', 'ADV')
3090 ('nebulous', 'ADJ')
3091 ('could', 'VERB')
3092 ('be', 'VERB')
3093 ('said', 'VERB')
3094 ('to', 'ADP')
3095 ('happen', 'VERB')
3096 ('.', '')
3097 ('It', 'PRON')
3098 ('was', 'VERB')
3099

...

11639 ('incidents', 'NOUN')
11640 ('without', 'ADP')
11641 ('being', 'VERB')
11642 ('able', 'ADJ')
11643 ('to', 'ADP')
11644 ('recapture', 'VERB')
11645 ('their', 'DET')
11646 ('atmosphere', 'NOUN')
11647 (',', '')
11648 ('and', 'CONJ')
11649 ('there', 'PRON')
11650 ('were', 'VERB')
11651 ('long', 'ADV')
11652 ('blank', 'ADJ')
11653 ('periods', 'NOUN')
11654 ('to', 'ADP')
11655 ('which', 'PRON')
11656 ('you', 'PRON')
11657 ('could', 'VERB')
11658 ('assign', 'VERB')
11659 ('nothing', 'PRON')
11660 ('.', '')
11661 ('Everything', 'PRON')
11662 ('had', 'VERB')
11663 ('been', 'VERB')
11664 ('different', 'ADJ')
11665 ('then', 'ADV')
11666 ('.', '')
11667 ('Even', 'ADV')
11668 ('the', 'DET')
11669 ('names', 'NOUN')
11670 ('of', 'ADP')
11671 ('countries', 'NOUN')
11672 (',', '')
11673 ('and', 'CONJ')
11674 ('their', 'DET')
11675 ('shapes', 'NOUN')
11676 ('on', 'ADP')
11677 ('the', 'DET')
11678 ('map', 'NOUN')
11679 (',', '')
11680 ('had', 'VERB')
11681 ('been', 'VERB')
11682 ('different', 'ADJ

12495 ('If', 'CONJ')
12496 ('the', 'DET')
12497 ('Party', 'NOUN')
12498 ('could', 'VERB')
12499 ('thrust', 'VERB')
12500 ('its', 'DET')
12501 ('hand', 'NOUN')
12502 ('into', 'ADP')
12503 ('the', 'DET')
12504 ('past', 'NOUN')
12505 ('and', 'CONJ')
12506 ('say', 'VERB')
12507 ('of', 'ADP')
12508 ('this', 'DET')
12509 ('or', 'CONJ')
12510 ('that', 'DET')
12511 ('event', 'NOUN')
12512 (',', '')
12513 ('it', 'PRON')
12514 ('never', 'ADV')
12515 ('happened', 'VERB')
12516 ('-', '')
12517 ('that', 'PRON')
12518 (',', '')
12519 ('surely', 'ADV')
12520 (',', '')
12521 ('was', 'VERB')
12522 ('more', 'ADV')
12523 ('terrifying', 'ADJ')
12524 ('than', 'ADP')
12525 ('mere', 'ADJ')
12526 ('torture', 'NOUN')
12527 ('and', 'CONJ')
12528 ('death', 'NOUN')
12529 ('?', '')
12530 ('The', 'DET')
12531 ('Party', 'NOUN')
12532 ('said', 'VERB')
12533 ('that', 'CONJ')
12534 ('Oceania', 'NOUN')
12535 ('had', 'VERB')
12536 ('never', 'ADV')
12537 ('been', 'VERB')
12538 ('in', 'ADP')
12539 ('alliance', 'NOUN')
1254

13408 ('bent', 'VERB')
13409 ('over', 'ADV')
13410 ('and', 'CONJ')
13411 ('tucked', 'VERB')
13412 ('the', 'DET')
13413 ('first', 'ADJ')
13414 ('joint', 'NOUN')
13415 ('of', 'ADP')
13416 ('her', 'DET')
13417 ('fingers', 'NOUN')
13418 ('under', 'ADP')
13419 ('her', 'DET')
13420 ('toes', 'NOUN')
13421 ('.', '')
13422 ('There', 'ADV')
13423 (',', '')
13424 ('comrades', 'NOUN')
13425 ('!', '')
13426 ('That', 'PRON')
13427 ("'s", 'VERB')
13428 ('how', 'ADV')
13429 ('I', 'PRON')
13430 ('want', 'VERB')
13431 ('to', 'ADP')
13432 ('see', 'VERB')
13433 ('you', 'PRON')
13434 ('doing', 'VERB')
13435 ('it', 'PRON')
13436 ('.', '')
13437 ('Watch', 'VERB')
13438 ('me', 'PRON')
13439 ('again', 'ADV')
13440 ('.', '')
13441 ('I', 'PRON')
13442 ("'m", 'VERB')
13443 ('thirty-nine', 'NUM')
13444 ('and', 'CONJ')
13445 ('I', 'PRON')
13446 ("'ve", 'VERB')
13447 ('had', 'VERB')
13448 ('four', 'NUM')
13449 ('children', 'NOUN')
13450 ('.', '')
13451 ('Now', 'CONJ')
13452 ('look', 'VERB')
13453 ('.', '')
13454 ('S

14185 ('Today', 'NOUN')
14186 ("'s", 'ADP')
14187 ('issue', 'NOUN')
14188 ('contained', 'VERB')
14189 ('a', 'DET')
14190 ('statement', 'NOUN')
14191 ('of', 'ADP')
14192 ('the', 'DET')
14193 ('actual', 'ADJ')
14194 ('output', 'NOUN')
14195 (',', '')
14196 ('from', 'ADP')
14197 ('which', 'PRON')
14198 ('it', 'PRON')
14199 ('appeared', 'VERB')
14200 ('that', 'CONJ')
14201 ('the', 'DET')
14202 ('forecasts', 'NOUN')
14203 ('were', 'VERB')
14204 ('in', 'ADP')
14205 ('every', 'DET')
14206 ('instance', 'NOUN')
14207 ('grossly', 'ADV')
14208 ('wrong', 'ADJ')
14209 ('.', '')
14210 ('Winston', 'NOUN')
14211 ("'s", 'ADP')
14212 ('job', 'NOUN')
14213 ('was', 'VERB')
14214 ('to', 'ADP')
14215 ('rectify', 'VERB')
14216 ('the', 'DET')
14217 ('original', 'ADJ')
14218 ('figures', 'NOUN')
14219 ('by', 'ADP')
14220 ('making', 'VERB')
14221 ('them', 'PRON')
14222 ('agree', 'VERB')
14223 ('with', 'ADP')
14224 ('the', 'DET')
14225 ('later', 'ADJ')
14226 ('ones', 'NOUN')
14227 ('.', '')
14228 ('As', 'CONJ')
1

15163 ('of', 'ADP')
15164 ('trying', 'VERB')
15165 ('to', 'ADP')
15166 ('keep', 'VERB')
15167 ('what', 'PRON')
15168 ('he', 'PRON')
15169 ('was', 'VERB')
15170 ('saying', 'VERB')
15171 ('a', 'DET')
15172 ('secret', 'NOUN')
15173 ('between', 'ADP')
15174 ('himself', 'PRON')
15175 ('and', 'CONJ')
15176 ('the', 'DET')
15177 ('telescreen', 'NOUN')
15178 ('.', '')
15179 ('He', 'PRON')
15180 ('looked', 'VERB')
15181 ('up', 'ADP')
15182 (',', '')
15183 ('and', 'CONJ')
15184 ('his', 'DET')
15185 ('spectacles', 'NOUN')
15186 ('darted', 'VERB')
15187 ('a', 'DET')
15188 ('hostile', 'ADJ')
15189 ('flash', 'NOUN')
15190 ('in', 'ADP')
15191 ('Winston', 'NOUN')
15192 ("'s", 'ADP')
15193 ('direction', 'NOUN')
15194 ('.', '')
15195 ('Winston', 'NOUN')
15196 ('hardly', 'ADV')
15197 ('knew', 'VERB')
15198 ('Tillotson', 'NOUN')
15199 (',', '')
15200 ('and', 'CONJ')
15201 ('had', 'VERB')
15202 ('no', 'DET')
15203 ('idea', 'NOUN')
15204 ('what', 'DET')
15205 ('work', 'NOUN')
15206 ('he', 'PRON')
15207 ('was

In [1058]:
## ----------- II ----------- #
#test label creator
bubu2=[]
dev_labels = []
tag_counter = 0
counter_label = 0
for idx3, k in enumerate(data):
    #print(idx3, k)
    for idx4, m in enumerate(k):
        #print(tag_counter,m)
        if (40*0)-1 < tag_counter < 40*100:
                bubu2.append(m[0])
                dev_labels.append(m[1])
                #print(ww[tag_counter])
                print(tag_counter,m)
        tag_counter += 1

0 ('It', 'PRON')
1 ('was', 'VERB')
2 ('a', 'DET')
3 ('bright', 'ADJ')
4 ('cold', 'ADJ')
5 ('day', 'NOUN')
6 ('in', 'ADP')
7 ('April', 'NOUN')
8 (',', '')
9 ('and', 'CONJ')
10 ('the', 'DET')
11 ('clocks', 'NOUN')
12 ('were', 'VERB')
13 ('striking', 'VERB')
14 ('thirteen', 'NUM')
15 ('.', '')
16 ('Winston', 'NOUN')
17 ('Smith', 'NOUN')
18 (',', '')
19 ('his', 'DET')
20 ('chin', 'NOUN')
21 ('nuzzled', 'VERB')
22 ('into', 'ADP')
23 ('his', 'DET')
24 ('breast', 'NOUN')
25 ('in', 'ADP')
26 ('an', 'DET')
27 ('effort', 'NOUN')
28 ('to', 'ADP')
29 ('escape', 'VERB')
30 ('the', 'DET')
31 ('vile', 'ADJ')
32 ('wind', 'NOUN')
33 (',', '')
34 ('slipped', 'VERB')
35 ('quickly', 'ADV')
36 ('through', 'ADP')
37 ('the', 'DET')
38 ('glass', 'NOUN')
39 ('doors', 'NOUN')
40 ('of', 'ADP')
41 ('Victory', 'NOUN')
42 ('Mansions', 'NOUN')
43 (',', '')
44 ('though', 'CONJ')
45 ('not', 'ADV')
46 ('quickly', 'ADV')
47 ('enough', 'ADV')
48 ('to', 'ADP')
49 ('prevent', 'VERB')
50 ('a', 'DET')
51 ('swirl', 'NOUN')
52

979 ('except', 'ADP')
980 ('a', 'DET')
981 ('series', 'NOUN')
982 ('of', 'ADP')
983 ('bright-lit', 'ADJ')
984 ('tableaux', 'NOUN')
985 ('occurring', 'VERB')
986 ('against', 'ADP')
987 ('no', 'DET')
988 ('background', 'NOUN')
989 ('and', 'CONJ')
990 ('mostly', 'ADV')
991 ('unintelligible', 'ADJ')
992 ('.', '')
993 ('Ministry', 'NOUN')
994 ('of', 'ADP')
995 ('Truth', 'NOUN')
996 (',', '')
997 ('-', '')
998 ('Minitrue', 'NOUN')
999 (',', '')
1000 ('in', 'ADP')
1001 ('Newspeak', 'NOUN')
1002 ('-', '')
1003 ('was', 'VERB')
1004 ('startlingly', 'ADV')
1005 ('different', 'ADJ')
1006 ('from', 'ADP')
1007 ('any', 'DET')
1008 ('other', 'DET')
1009 ('object', 'NOUN')
1010 ('in', 'ADP')
1011 ('sight', 'NOUN')
1012 ('.', '')
1013 ('It', 'PRON')
1014 ('was', 'VERB')
1015 ('an', 'DET')
1016 ('enormous', 'ADJ')
1017 ('pyramidal', 'ADJ')
1018 ('structure', 'NOUN')
1019 ('of', 'ADP')
1020 ('glittering', 'ADJ')
1021 ('white', 'ADJ')
1022 ('concrete', 'NOUN')
1023 (',', '')
1024 ('soaring', 'VERB')
1025 (

1846 ('of', 'ADP')
1847 ('a', 'DET')
1848 ('frowsy', 'ADJ')
1849 ('little', 'ADJ')
1850 ('junk-shop', 'NOUN')
1851 ('in', 'ADP')
1852 ('a', 'DET')
1853 ('slummy', 'ADJ')
1854 ('quarter', 'NOUN')
1855 ('of', 'ADP')
1856 ('the', 'DET')
1857 ('town', 'NOUN')
1858 ('(', '')
1859 ('just', 'ADV')
1860 ('what', 'DET')
1861 ('quarter', 'NOUN')
1862 ('he', 'PRON')
1863 ('did', 'VERB')
1864 ('not', 'ADV')
1865 ('now', 'ADV')
1866 ('remember', 'VERB')
1867 (')', '')
1868 ('and', 'CONJ')
1869 ('had', 'VERB')
1870 ('been', 'VERB')
1871 ('stricken', 'VERB')
1872 ('immediately', 'ADV')
1873 ('by', 'ADP')
1874 ('an', 'DET')
1875 ('overwhelming', 'ADJ')
1876 ('desire', 'NOUN')
1877 ('to', 'ADP')
1878 ('possess', 'VERB')
1879 ('it', 'PRON')
1880 ('.', '')
1881 ('Party', 'NOUN')
1882 ('members', 'NOUN')
1883 ('were', 'VERB')
1884 ('supposed', 'VERB')
1885 ('not', 'ADV')
1886 ('to', 'ADP')
1887 ('go', 'VERB')
1888 ('into', 'ADP')
1889 ('ordinary', 'ADJ')
1890 ('shops', 'NOUN')
1891 ('(', '')
1892 ('dealin

2646 ('Last', 'ADJ')
2647 ('night', 'NOUN')
2648 ('to', 'ADP')
2649 ('the', 'DET')
2650 ('flicks', 'NOUN')
2651 ('.', '')
2652 ('All', 'DET')
2653 ('war', 'NOUN')
2654 ('films', 'NOUN')
2655 ('.', '')
2656 ('One', 'NUM')
2657 ('very', 'ADV')
2658 ('good', 'ADJ')
2659 ('one', 'PRON')
2660 ('of', 'ADP')
2661 ('a', 'DET')
2662 ('ship', 'NOUN')
2663 ('full', 'ADJ')
2664 ('of', 'ADP')
2665 ('refugees', 'NOUN')
2666 ('being', 'VERB')
2667 ('bombed', 'VERB')
2668 ('somewhere', 'ADV')
2669 ('in', 'ADP')
2670 ('the', 'DET')
2671 ('Mediterranean', 'NOUN')
2672 ('.', '')
2673 ('Audience', 'NOUN')
2674 ('much', 'ADV')
2675 ('amused', 'ADJ')
2676 ('by', 'ADP')
2677 ('shots', 'NOUN')
2678 ('of', 'ADP')
2679 ('a', 'DET')
2680 ('great', 'ADJ')
2681 ('huge', 'ADJ')
2682 ('fat', 'ADJ')
2683 ('man', 'NOUN')
2684 ('trying', 'VERB')
2685 ('to', 'ADP')
2686 ('swim', 'VERB')
2687 ('away', 'ADV')
2688 ('with', 'ADP')
2689 ('a', 'DET')
2690 ('helicopter', 'NOUN')
2691 ('after', 'ADP')
2692 ('him', 'PRON')
2693

3211 ('since', 'CONJ')
3212 ('he', 'PRON')
3213 ('had', 'VERB')
3214 ('sometimes', 'ADV')
3215 ('seen', 'VERB')
3216 ('her', 'PRON')
3217 ('with', 'ADP')
3218 ('oily', 'ADJ')
3219 ('hands', 'NOUN')
3220 ('and', 'CONJ')
3221 ('carrying', 'VERB')
3222 ('a', 'DET')
3223 ('spanner', 'NOUN')
3224 ('she', 'PRON')
3225 ('had', 'VERB')
3226 ('some', 'DET')
3227 ('mechanical', 'ADJ')
3228 ('job', 'NOUN')
3229 ('on', 'ADP')
3230 ('one', 'PRON')
3231 ('of', 'ADP')
3232 ('the', 'DET')
3233 ('novel-writing', 'ADJ')
3234 ('machines', 'NOUN')
3235 ('.', '')
3236 ('She', 'PRON')
3237 ('was', 'VERB')
3238 ('a', 'DET')
3239 ('bold-looking', 'ADJ')
3240 ('girl', 'NOUN')
3241 (',', '')
3242 ('of', 'ADP')
3243 ('about', 'ADV')
3244 ('twenty-seven', 'NUM')
3245 (',', '')
3246 ('with', 'ADP')
3247 ('thick', 'ADJ')
3248 ('hair', 'NOUN')
3249 (',', '')
3250 ('a', 'DET')
3251 ('freckled', 'ADJ')
3252 ('face', 'NOUN')
3253 (',', '')
3254 ('and', 'CONJ')
3255 ('swift', 'ADJ')
3256 (',', '')
3257 ('athletic', 'ADJ

3998 ('the', 'DET')
3999 ('Party', 'NOUN')


The below code checks whether the two match

In [1552]:
for idx, x in enumerate(extra_set_list):
    if not x == bubu2[idx]:
        print(x, bubu2[idx], idx)
#      #break
#print(extra_set_list[1190:1200])
#print(bubu2[1190:1200])
print(extra_set_list[10600:10610])
print(dev_labels[10600:10610])
print(tens[10600][0])
print(tens[10601][0])
xx = np.copy(tens[0:6600])
yy = np.copy(dev_labels[0:6600])

Winston's Winston 510
people's people 576
Winston's Winston 594
tomorrow's tomorrow 1392
child's child 2883
O'Brien's O'Brien 3663
prize-fighter's prize-fighter 3668
O'Brien's O'Brien 3694
one's one 3894
one's one 3906
Party's Party 4058
Winston's Winston 4128
Goldstein's Goldstein 4403
Goldstein's Goldstein 4468
O'Brien's O'Brien 4867
Goldstein's Goldstein 4934
one's one 5055
Winston's Winston 5098
one's one 5276
one's one 5299
sheep's sheep 5470
everyone's everyone 5691
. ... 5779
. ... 5784
. ... 5933
O'Brien's O'Brien 6027
O'Brien's O'Brien 6181
else's else 6188
it's it 7375
Winston's Winston 7394
It's It 7572
he'd he 7676
He's He 7693
Winston's Winston 7706
I'm I 7947
boy's boy 8119
You're You 8130
You're You 8138
You're You 8142
I'll I 8147
I'll I 8151
I'll I 8155
boy's boy 8216
They're They 8307
that's that 8318
woman's woman 8522
O'Brien's O'Brien 9011
morning's morning 9046
,'t , 9303
,'t , 9327
father's father 10355
one's one 10681
mother's mother 10783
women's women 11021
3 

he remember 11755
did his 11756
remember father 11757
his hand 11758
father's clutching 11759
hand his 11760
clutching own 11761
his as 11762
own they 11763
as hurried 11764
they down 11765
hurried , 11766
, into 11770
down some 11771
into place 11772
some deep 11773
place in 11774
deep the 11775
in earth 11776
the , 11777
earth round 11778
, and 11779
and a 11781
round spiral 11782
a staircase 11783
spiral which 11784
staircase rang 11785
which under 11786
rang his 11787
under feet 11788
his and 11789
feet which 11790
and finally 11791
which so 11792
finally wearied 11793
so his 11794
wearied legs 11795
his that 11796
legs he 11797
that began 11798
he whimpering 11799
began and 11800
whimpering they 11801
and had 11802
they to 11803
had stop 11804
to and 11805
stop rest 11806
and . 11807
rest His 11808
. mother 11809
His , 11810
mother in 11811
, her 11812
in slow 11813
her , 11814
slow dreamy 11815
, way 11816
dreamy , 11817
way was 11818
, following 11819
was a 11820
following long 

true If 12445
. the 12446
If Party 12447
the could 12448
Party thrust 12449
could its 12450
thrust hand 12451
its into 12452
hand the 12453
into past 12454
the and 12455
past say 12456
and of 12457
say this 12458
of or 12459
this that 12460
or event 12461
that , 12462
event it 12463
, never 12464
it happened 12465
never - 12466
happened that 12467
- , 12468
that surely 12469
surely was 12471
, more 12472
was terrifying 12473
more than 12474
terrifying mere 12475
than torture 12476
mere and 12477
torture death 12478
and ? 12479
death The 12480
? Party 12481
The said 12482
Party that 12483
said Oceania 12484
that had 12485
Oceania never 12486
had been 12487
never in 12488
been alliance 12489
in with 12490
alliance Eurasia 12491
with . 12492
Eurasia He 12493
. , 12494
He Winston 12495
, Smith 12496
Winston , 12497
Smith knew 12498
, that 12499
knew Oceania 12500
that had 12501
Oceania been 12502
had in 12503
been alliance 12504
in with 12505
alliance Eurasia 12506
with as 12507
Eurasia sh

much this 13082
of legend 13083
this was 13084
legend true 13085
was and 13086
true how 13087
and much 13088
how invented 13089
much . 13090
invented Winston 13091
. could 13092
Winston not 13093
could even 13094
not remember 13095
even at 13096
remember what 13097
at date 13098
what the 13099
date Party 13100
the itself 13101
Party had 13102
itself come 13103
had into 13104
come existence 13105
into . 13106
existence He 13107
. did 13108
He not 13109
did believe 13110
not he 13111
believe had 13112
he ever 13113
had heard 13114
ever the 13115
heard word 13116
the Ingsoc 13117
word before 13118
Ingsoc 1960 13119
before , 13120
1960 but 13121
, it 13122
but was 13123
it possible 13124
was that 13125
possible in 13126
that its 13127
in Oldspeak 13128
its form 13129
Oldspeak - 13130
form English 13131
- Socialism 13132
English , 13133
Socialism that 13134
, is 13135
that to 13136
is say 13137
to - 13138
say it 13139
- had 13140
it been 13141
had current 13142
been earlier 13143
current . 

the warning 14267
end that 14268
of it 14269
the would 14270
present probably 14271
week be 14272
. necessary 14273
All to 14274
that reduce 14275
was the 14276
needed ration 14277
was at 14278
to some 14279
substitute time 14280
for in 14281
the April 14282
original . 14283
promise As 14284
a soon 14285
warning as 14286
that Winston 14287
it had 14288
would dealt 14289
probably with 14290
be each 14291
necessary of 14292
to the 14293
reduce messages 14294
the , 14295
ration he 14296
at clipped 14297
some his 14298
time speakwritten 14299
in corrections 14300
April to 14301
. the 14302
As appropriate 14303
soon copy 14304
as of 14305
Winston the 14306
had Times 14307
dealt and 14308
with pushed 14309
each them 14310
of into 14311
messages pneumatic 14313
, tube 14314
he . 14315
clipped Then 14316
his , 14317
speakwritten with 14318
corrections a 14319
to movement 14320
the which 14321
appropriate was 14322
copy as 14323
of nearly 14324
the as 14325
Times possible 14326
and unconscious 

very was 15105
close saying 15106
to a 15107
the secret 15108
mouthpiece between 15109
of himself 15110
the and 15111
speakwrite the 15112
. telescreen 15113
He . 15114
had He 15115
the looked 15116
air up 15117
of , 15118
trying and 15119
to his 15120
keep spectacles 15121
what darted 15122
he a 15123
was hostile 15124
saying flash 15125
a in 15126
secret Winston 15127
between direction 15128
himself . 15129
and Winston 15130
the hardly 15131
telescreen knew 15132
. Tillotson 15133
He , 15134
looked and 15135
up had 15136
, no 15137
and idea 15138
his what 15139
spectacles work 15140
darted he 15141
a was 15142
hostile employed 15143
flash on 15144
in . 15145
Winston's People 15146
direction in 15147
. the 15148
Winston Records 15149
hardly Department 15150
knew did 15151
Tillotson not 15152
, readily 15153
and talk 15154
had about 15155
no their 15156
idea jobs 15157
what . 15158
work In 15159
he the 15160
was long 15161
employed , 15162
on windowless 15163
. hall 15164
People , 1516

sent was 15789
out permitted 15790
in to 15791
sealed look 15792
packets at 15793
and . 15794
which Three 15795
no messages 15796
Party had 15797
member slid 15798
, out 15799
other of 15800
than the 15801
those pneumatic 15802
who tube 15803
worked while 15804
on Winston 15805
it was 15806
, working 15807
was , 15808
permitted but 15809
to they 15810
look were 15811
at simple 15812
. matters 15813
Three , 15814
messages and 15815
had he 15816
slid had 15817
out disposed 15818
the them 15820
pneumatic before 15821
tube the 15822
while Two 15823
Winston Minutes 15824
was Hate 15825
working interrupted 15826
, him 15827
but . 15828
they When 15829
were the 15830
simple Hate 15831
matters was 15832
, over 15833
and he 15834
he returned 15835
had to 15836
disposed his 15837
of cubicle 15838
them , 15839
before took 15840
Two Newspeak 15842
Minutes Dictionary 15843
Hate from 15844
interrupted the 15845
him shelf 15846
. , 15847
When pushed 15848
Hate speakwrite 15850
was to 15851
over one 1

In [1544]:
#xx[~np.isnan(xx).any(axis=1), :]
#yy[~np.isnan(xx).any(), :]

#print(np.argwhere(np.isnan(xx[:,0])))
#print(np.shape(xx))
print(np.shape(xx[~np.isnan(xx).any(axis=1),:]))
print(np.shape(yy[~np.isnan(xx[:,0])]))
# [[31. 32. 33. 34.]]
print(np.shape(xx))
print(np.shape(yy))
yy = yy[~np.isnan(xx[:,0])]
xx = xx[~np.isnan(xx).any(axis=1),:]

(6585, 768)
(6585,)
(6600, 768)
(6600,)


In [326]:
# tens2 = np.copy(tens)
# ww2 = np.copy(ww)
# extra_set_list2 = np.copy(extra_set_list)
# dev_labels2 = np.copy(dev_labels)

In [1545]:
#tenscontextbackup=np.copy(tens)
#tenscontextlabels=np.copy(dev_labels)
#tensbackup = np.copy(tens)
#wwbackup = np.copy(ww)
#extra_set_list_backup = np.copy(extra_set_list)
#tensbackup_finetunedPOS = np.copy(tens)
#extra_set_list_backup_finetunedPOS = np.copy(extra_set_list)
#labels_backup = np.copy(dev_labels)
print(len(ww))
print(np.shape(xx))
#print(np.shape(X_scaled))
print(np.shape(dev_labels))
fun_train = xx #np.concatenate((tens2,tensbackup))
fun_label =  yy #np.concatenate((dev_labels2,labels_backup))
np.shape(fun_train),np.shape(fun_label)

40
(6585, 768)
(15972,)


((6585, 768), (6585,))

In [1546]:
from sklearn.model_selection import train_test_split
TEST_SIZE = 0.1
X_train, X_test, Y_train, Y_test = train_test_split(fun_train, fun_label, test_size=TEST_SIZE, shuffle=False,random_state=2)

In [1547]:
Y_traini, Y_testi = label_encoder(Y_train,Y_test)

In [1548]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X_train)
X_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [1549]:
clf = LogisticRegression(max_iter=1000000)
clf.fit(X_scaled, Y_train)
dev_pred_labels = clf.predict(X_test_scaled)

print('train_acc',clf.score(X_scaled, Y_train))
print('test_acc',clf.score(X_test_scaled,Y_test))

#print('acc', accuracy_score(Y_test ,dev_pred_labels))
#For partial dataset:
# train_acc 0.9802
# test_acc 0.4919735599622285
# for whole dataset:
#train_acc 0.7923755513547575
#test_acc 0.5873465533522191
#with standard scaling:
#train_acc 0.7864944339424491
#test_acc 0.6005665722379604
#and no shuffling
#train_acc 0.7827137156059651
#test_acc 0.6203966005665722

train_acc 0.8889638879514006
test_acc 0.5371775417298937


In [1500]:
#import pandas
#import seaborn
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test,dev_pred_labels)

array([[167,   7,   2,  10,   7,  12,  14,   0,  12,   9,   0],
       [  8,  70,  19,   3,   7,   6,  16,   0,   3,  13,   0],
       [ 17,  14, 153,  11,   8,  24,  24,   3,  11,  23,   0],
       [  9,   5,  12,  58,  10,   6,   8,   0,   4,  10,   0],
       [ 11,   4,   6,   7,  58,   8,   7,   0,   6,  12,   0],
       [ 19,   2,  13,   6,   6, 153,  39,   2,   6,  11,   0],
       [ 26,  15,  32,   5,  10,  31, 256,   3,   6,  31,   0],
       [  4,   1,   1,   1,   0,   0,   2,   5,   1,   0,   0],
       [  8,   7,   7,   6,   4,  10,   9,   0,  98,  21,   0],
       [ 34,  12,  14,  18,  12,   9,  21,   0,  25, 199,   0],
       [  0,   0,   0,   0,   0,   1,   0,   0,   1,   0,   0]])

In [1539]:
print(np.shape(X_train))
print(np.shape(X_test))

print(np.shape(Y_traini))
print(np.shape(Y_testi))

#X_trainz = X_train[0:10750,:]
#Y_trainz = Y_train[0:10750]
#X_testz = X_test[0:1190,:]
#Y_testz = Y_test[0:1190]

#X_trainz = np.reshape(X_trainz,(1075,10,768))
#Y_trainz = np.reshape(Y_trainz,(1075,10,11))
#X_testz = np.reshape(X_testz,(119,10,768))
#Y_testz = np.reshape(Y_testz,(119,10,11))


(5926, 768)
(659, 768)
(5926, 10)
(659, 10)


In [1467]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, Dense, TimeDistributed, Dropout, Activation,SimpleRNN, LSTM

model = Sequential ()
#model.add(Bidirectional(LSTM(64, return_sequences = True)))
model.add(Dense(768, activation='linear'))
#model.add(Dropout(0.2))
#model.add(Bidirectional(LSTM(32, return_sequences = True)))
#model.add(Dense(11, activation='sigmoid'))
#model.add(Dropout(0.4))
#model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(Dense(10, activation='softmax'))
#model.add(TimeDistributed(Dense(11, activation='softmax')))



In [1468]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [1469]:
model.build((4736, 768))
model.summary()

Model: "sequential_74"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_140 (Dense)            (4736, 768)               590592    
_________________________________________________________________
dense_141 (Dense)            (4736, 10)                7690      
Total params: 598,282
Trainable params: 598,282
Non-trainable params: 0
_________________________________________________________________


In [1523]:
history = model.fit(X_train, Y_traini, batch_size=32, epochs=400, validation_data=(X_test,Y_testi))

Epoch 1/400


ValueError: in user code:

    /Users/Mahwish/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /Users/Mahwish/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/Mahwish/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/Mahwish/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/Mahwish/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/Mahwish/anaconda3/lib/python3.7/site-packages/t

In [1456]:
# Xnew= X_train[-500:]
# ynew = np.argmax(model.predict(Xnew), axis=-1)

# for i in range(len(Xnew)):
# 	print("Predicted=%s, Actual=%s" % ( ynew[i],Y_traini[-500:][i]))
 
# #print(ynew)

In [495]:
from keras.utils import np_utils

In [1063]:
from sklearn.preprocessing import LabelEncoder
def label_encoder(train_labels, test_labels):
	le = LabelEncoder()
	le.fit(np.concatenate((train_labels,test_labels)))
    
    #transform
	#train_label_enc = le.transform(train_labels)
	#test_label_enc = le.transform(test_labels)

    #transform + one-hot
	train_label_enc = np_utils.to_categorical(le.transform(train_labels))
	test_label_enc = np_utils.to_categorical(le.transform(test_labels))
    
	return train_label_enc, test_label_enc

In [1393]:
Y_traini, Y_testi = label_encoder(Y_train,Y_test)

#test_label_enc_backup

In [1193]:
#history = model.fit(X_scaled,train_label_enc, epochs=10, batch_size=32, validation_data=(X_test_scaled,test_label_enc))

In [1457]:
#historycopy= np.copy(history) #dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])
history.history.keys()
#history.history['accuracy']

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('Number of Epochs')
plt.ylabel('Loss')
plt.legend(['training','validation'])
plt.savefig('Bidirec_POS_loss_graph')

NameError: name 'plt' is not defined

In [1229]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
def build_model(input_dim, hidden_neurons, output_dim):
    """
    Construct, compile and return a Keras model which will be used to fit/predict
    """
    model = Sequential([
        Dense(hidden_neurons, input_dim=input_dim),
        Activation('relu'),
        Dropout(0.2),
        Dense(hidden_neurons),
        Activation('sigmoid'),
        Dropout(0.2),
        Dense(hidden_neurons),
        Activation('sigmoid'),
        Dropout(0.2),
        Dense(hidden_neurons),
        Activation('relu'),
        Dense(output_dim, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [1230]:
#np.shape(train_label_enc[1])

In [1231]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
model_params = {
    'build_fn': build_model,
    'input_dim': X_scaled.shape[1],
    'hidden_neurons': 512,
    'output_dim': train_label_enc.shape[1],
    'epochs': 100,
    'batch_size': 456,
    'verbose': 1,
    'validation_data': (X_test_scaled, test_label_enc),
    'shuffle': True
}
clf = KerasClassifier(**model_params)

In [1237]:
X_scaled.shape
#train_label_enc.shape

(1799, 1280)

In [1236]:
hist = clf.fit(X_scaled, train_label_enc)

Epoch 1/100
4/4 [==============================] - 2s 164ms/step - loss: 2.5042 - accuracy: 0.1077 - val_loss: 2.3024 - val_accuracy: 0.1435
Epoch 2/100
4/4 [==============================] - 0s 101ms/step - loss: 2.3076 - accuracy: 0.1503 - val_loss: 2.1842 - val_accuracy: 0.2225
Epoch 3/100
4/4 [==============================] - 0s 110ms/step - loss: 2.2558 - accuracy: 0.1691 - val_loss: 2.1542 - val_accuracy: 0.1446
Epoch 4/100
4/4 [==============================] - 0s 99ms/step - loss: 2.2188 - accuracy: 0.1523 - val_loss: 2.1557 - val_accuracy: 0.1446
Epoch 5/100
4/4 [==============================] - 0s 99ms/step - loss: 2.1996 - accuracy: 0.1660 - val_loss: 2.1480 - val_accuracy: 0.2214
Epoch 6/100
4/4 [==============================] - 0s 95ms/step - loss: 2.1875 - accuracy: 0.1690 - val_loss: 2.1398 - val_accuracy: 0.1446
Epoch 7/100
4/4 [==============================] - 0s 99ms/step - loss: 2.1594 - accuracy: 0.1996 - val_loss: 2.1306 - val_accuracy: 0.2225
Epoch 8/100
4/4 [

Epoch 59/100
4/4 [==============================] - 0s 104ms/step - loss: 0.3124 - accuracy: 0.8895 - val_loss: 5.6985 - val_accuracy: 0.1546
Epoch 60/100
4/4 [==============================] - 0s 103ms/step - loss: 0.2766 - accuracy: 0.9103 - val_loss: 5.8205 - val_accuracy: 0.1591
Epoch 61/100
4/4 [==============================] - 0s 117ms/step - loss: 0.2433 - accuracy: 0.9219 - val_loss: 5.8199 - val_accuracy: 0.1502
Epoch 62/100
4/4 [==============================] - 0s 128ms/step - loss: 0.2541 - accuracy: 0.9106 - val_loss: 5.9129 - val_accuracy: 0.1613
Epoch 63/100
4/4 [==============================] - 0s 122ms/step - loss: 0.2193 - accuracy: 0.9280 - val_loss: 6.0657 - val_accuracy: 0.1580
Epoch 64/100
4/4 [==============================] - 1s 242ms/step - loss: 0.2138 - accuracy: 0.9370 - val_loss: 6.0368 - val_accuracy: 0.1591
Epoch 65/100
4/4 [==============================] - 0s 98ms/step - loss: 0.2221 - accuracy: 0.9321 - val_loss: 6.3569 - val_accuracy: 0.1491
Epoch 6

In [217]:
from numpy import savez_compressed

In [226]:
#savez_compressed('BERT_tens' ,tens)
#savez_compressed('BERT_labels' ,dev_labels)
#savez_compressed('BERT_extra_set_list', extra_set_list)
#savez_compressed('BERT_bubu2', bubu2)

In [338]:
#savez_compressed('BERT_tens_total' ,fun_train)
#savez_compressed('BERT_labels_total' ,fun_labels)
#savez_compressed('BERT_extra_set_list_total', extra_set_list)
#savez_compressed('BERT_bubu2_total', bubu2)

In [228]:
np.shape(tens)

torch.Size([11564, 768])